In [ ]:
!pip install transformers pandas scikit-learn torch

In [2]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
def batch_tokenize(texts, labels, tokenizer, batch_size=10000):
    all_encodings = {"input_ids": [], "attention_mask": []}
    all_labels = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        batch_labels = labels[i:i+batch_size]
        batch_enc = tokenizer(batch_texts, truncation=True, padding=True, return_tensors="pt")
        all_encodings["input_ids"].extend(batch_enc["input_ids"])
        all_encodings["attention_mask"].extend(batch_enc["attention_mask"])
        all_labels.extend(batch_labels)
    return all_encodings, all_labels

class PhishingDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        return {
            key: self.encodings[key][idx] for key in self.encodings
        } | {"labels": torch.tensor(self.labels[idx])}
    def __len__(self):
        return len(self.labels)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }


In [ ]:
dataset_path = "/content/drive/MyDrive/phishing_chunks/phishing_urls_part1.csv"
df = pd.read_csv(dataset_path)

train_texts, test_texts, train_labels, test_labels = train_test_split(
    df["url"].tolist(), df["label"].tolist(),
    test_size=0.2, stratify=df["label"], random_state=42
)

In [12]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
train_encodings, train_labels_final = batch_tokenize(train_texts, train_labels, tokenizer)
test_encodings, test_labels_final = batch_tokenize(test_texts, test_labels, tokenizer)
train_dataset = PhishingDataset(train_encodings, train_labels_final)
test_dataset = PhishingDataset(test_encodings, test_labels_final)

In [13]:
model_path = "/content/drive/MyDrive/phishing_model_checkpoint"
try:
    model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=2)
except:
    model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
training_args = TrainingArguments(
    output_dir=model_path,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="/content/drive/MyDrive/phishing_logs",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=1,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    save_total_limit=2,
    logging_steps=50,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    data_collator=data_collator
)


In [ ]:
trainer.train()
trainer.evaluate()

In [10]:
trainer.save_model("/content/drive/MyDrive/phishing_model_v1_after_phase1")